In [136]:
import pandas as pd
from colorthief import ColorThief
import io
import urllib
import os
import re
import colorgram
import numpy as np

In [137]:
# import raw data
artist = pd.read_csv('../raw_data/artist_data.csv')
artwork = pd.read_csv('../raw_data/artwork_data.csv', dtype={'creditLine': str, 'height': str, 'year': str})

In [138]:
# graph 1: # artists artwork, gender in Tate collection
# maybe for future: age @ first Tate acquisition?

# merge artist and artwork on artist id
artists_artwork = artist.merge(artwork, 'left', left_on = 'id', right_on = 'artistId')
aa_counts = artists_artwork.groupby('id_x')['name'].count()
aa_counts = aa_counts.to_frame().rename(columns = {'name': 'artwork_count'}).reset_index()

# merge in gender
aa_counts = aa_counts.merge(artist, left_on='id_x', right_on='id')
aa_counts = aa_counts[['id', 'name', 'artwork_count', 'gender']]

# # save to csv
# aa_counts.to_csv('../cleaned_data/aa_counts.csv')

In [88]:
# graph 1: artists artwork (1 each)
def get_pics1(row):
    ''' 
    Takes row of dataframe & downloads image from url
    '''
    id, _, _, _, _, url = row
    print(id)
    if type(url) == str:
        urllib.request.urlretrieve(url, f'/Users/carolynliu/CAPP30239_FA22/final_project/pics1/{id}.jpg')

In [139]:
# graph 1 get pics
df = pd.read_csv('../raw_data/the-tate-collection.csv', sep=';')
df = df[['artistId', 'thumbnailUrl']]
df.drop_duplicates('artistId', inplace=True)

df1 = aa_counts.merge(df, left_on='id', right_on='artistId')
# df1.apply(lambda x: get_pics1(x), axis = 1)

In [109]:
df1

,id,name,artwork_count,gender,artistId,thumbnailUrl,img
0,0,"Abbey, Edwin Austin",6,Male,0,https://public.opendatasoft.com/api/v2/catalog...,/Users/carolynliu/CAPP30239_FA22/final_project...
1,1,"Abbott, Lemuel Francis",2,Male,1,https://public.opendatasoft.com/api/v2/catalog...,/Users/carolynliu/CAPP30239_FA22/final_project...
2,2,"Agasse, Jacques Laurent",2,Male,2,https://public.opendatasoft.com/api/v2/catalog...,/Users/carolynliu/CAPP30239_FA22/final_project...
3,3,"Alken, Henry Thomas",5,Male,3,https://public.opendatasoft.com/api/v2/catalog...,/Users/carolynliu/CAPP30239_FA22/final_project...
4,4,"Allan, Sir William",1,Male,4,https://public.opendatasoft.com/api/v2/catalog...,/Users/carolynliu/CAPP30239_FA22/final_project...
...,...,...,...,...,...,...,...
3333,17412,"Whitten, Jack",1,Male,17412,NaN,
3334,17414,"Fukushima, Hideko",1,Female,17414,https://public.opendatasoft.com/api/v2/catalog...,/Users/carolynliu/CAPP30239_FA22/final_project...
3335,17422,"Wylie, Rose",1,Female,17422,NaN,
3336,17450,"Crozier, William",1,Male,17450,NaN,


In [140]:
df1['img'] = df1.apply(lambda x: '' if type(x['thumbnailUrl'])==float else f'/Users/carolynliu/CAPP30239_FA22/final_project/pics1/{x["id"]}.jpg', axis = 1)
df1.drop(['artistId', 'thumbnailUrl'], axis = 1, inplace=True)
df1['parent'] = 'parent'
df1.to_csv('../cleaned_data/aa_counts_url.csv')

In [4]:
# graph 2: # Tate acquisition over time & medium?

acquisitions = artwork.groupby(['acquisitionYear'])['acquisitionYear'].count()
acquisitions = acquisitions.to_frame().rename(columns = {'acquisitionYear': 'ac_count'}).reset_index()

# most popular medium by acquisition year
medium = artwork.groupby(['acquisitionYear', 'medium'])['medium'].count()
medium = medium.to_frame().rename(columns = {'medium': 'med_count'}).reset_index()
medium = medium.groupby('acquisitionYear', group_keys=False).apply(lambda x: x.loc[x.med_count.idxmax()]).reset_index(drop=True)

# merge acquisitions and medium
ac_med = acquisitions.merge(medium)

#save to csv
ac_med.to_csv('../cleaned_data/ac_med.csv')

In [20]:
# graph 3 - colors
# keep only watercolors
wc = artwork[artwork['medium'].str.contains("watercolour", case = False, na=False)]
wc.to_csv('../raw_data/watercolours.csv')

In [40]:
# data for artwork thumbnails from opendatasoft - https://public.opendatasoft.com/explore/dataset/the-tate-collection/table/
df = pd.read_csv('../raw_data/the-tate-collection-3.csv', sep=';')
df = df[['accession_number', 'thumbnailUrl']]

In [49]:
#merge w/ watercolors
wc1 = wc.merge(df, on = 'accession_number')
wc2 = wc1[['accession_number', 'thumbnailUrl_y']]
wc2.dropna(inplace=True)

/var/folders/bq/y6jssnk56235t8wjc47m1fdw0000gn/T/ipykernel_72483/1604263833.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wc2.dropna(inplace=True)


In [56]:
url = 'https://public.opendatasoft.com/api/v2/catalog/datasets/the-tate-collection/files/3fa7321cc3eac4b92b2409fcf5c207a4'
urllib.request.urlretrieve(url, f'/Users/carolynliu/CAPP30239_FA22/final_project/pics/test.jpg')

('/Users/carolynliu/CAPP30239_FA22/final_project/pics/test.jpg',
 <http.client.HTTPMessage at 0x14b39ceb0>)

In [75]:
def get_pics(row):
    ''' 
    Takes row of dataframe & downloads image from url
    '''
    num, url = row
    print(num)
    urllib.request.urlretrieve(url, f'/Users/carolynliu/CAPP30239_FA22/final_project/pics/{num}.jpg')

In [76]:
wc2.apply(lambda x: get_pics(x), axis = 1)
# got thru D01659

A00001
A00034
A00041
A00061
A00062
A00129
A00130
A00168
A00169
A00170
A00171
A00172
A00173
A00174
A00175
A00176
A00177
A00178
A00179
A00180
A00181
A00182
A00183
A00184
A00185
A00186
A00187
A00188
A00189
A00194
A00195
A00202
A00205
A00221
A00223
A00225
A00226
A00228
A00229
A00244
A00248
A00250
A00254
A00268
A00269
A00270
A00290
A00292
A00313
A00335
A00344
A00351
A00354
A00356
A00360
A00368
A00372
A00373
A00374
A00382
A00388
A00402
A00409
A00449
A00450
A00454
A00456
A00458
A00460
A00462
A00465
A00468
A00469
A00470
A00472
A00473
A00474
A00475
A00476
A00478
A00479
A00481
A00482
A00483
A00485
A00487
A00488
A00489
A00490
A00492
A00494
A00495
A00496
A00497
A00498
A00499
A00502
A00503
A00504
A00506
A00508
A00510
A00511
A00512
A00513
A00515
A00516
A00518
A00519
A00521
A00529
A00531
A00544
A00546
A00556
A00557
A00559
A00561
A00562
A00563
A00564
A00565
A00566
A00567
A00568
A00569
A00570
A00571
A00572
A00573
A00574
A00575
A00576
A00577
A00578
A00579
A00580
A00581
A00582
A00584
A00586
A00587
A00588

URLError: <urlopen error [Errno 60] Operation timed out>

In [88]:
# take out rows already have images for & download images
wc3 = wc2.loc[wc2.loc[wc2['accession_number'] == 'D01660'].index[0]: ,]
wc3.apply(lambda x: get_pics(x), axis = 1)

D01660
D01661
D01668
D01669
D01670
D01671
D01672
D01673
D01674
D01675
D01862
D01873
D01878
D01879
D01880
D01882
D01883
D01891
D01892
D01894
D01900
D01901
D01902
D01903
D01904
D01905
D01906
D01907
D01908
D01909
D01910
D01911
D01920
D02187
D02188
D02223
D02224
D02238
D02240
D02241
D02342
D02343
D02344
D02345
D02346
D02347
D02348
D02350
D02352
D02353
D02354
D02357
D02374
D02375
D02377
D02378
D02383
D02391
D02474
D03382
D03383
D03384
D03386
D03387
D03389
D03405
D03408
D03413
D03414
D03415
D03420
D03421
D03422
D03423
D03424
D03425
D03426
D03429
D03431
D03435
D03436
D03632
D03633
D03634
D03635
D03636
D03637
D03638
D03639
D03640
D03641
D03642
D03643
D03644
D03647
D03648
D03649
D03650
D03651
D03652
D04152
D04155
D04158
D04159
D04160
D04161
D04162
D04163
D04164
D04165
D04166
D04167
D04168
D04169
D04172
D04174
D04175
D04176
D04177
D04178
D04179
D04180
D04181
D04182
D04183
D04185
D04186
D04187
D04397
D04427
D04460
D04495
D04496
D04497
D04500
D04501
D04502
D04513
D04525
D04526
D04527
D04599
D04601

932     None
933     None
934     None
935     None
936     None
        ... 
7330    None
7331    None
7332    None
7333    None
7346    None
Length: 6173, dtype: object

In [74]:
# Get dominant (color thief) color for watercolor artworks
dir = '/Users/carolynliu/CAPP30239_FA22/final_project/pics'
colors = {}
for filename in os.listdir(dir):
    exp = re.compile(".jpg")
    if re.search(exp, filename):
        f = os.path.join(dir, filename)
        color_thief = ColorThief(f)
        # get the dominant color
        dominant_color = color_thief.get_color(quality=1)
        #color_thief.get_palette(color_count=6)
        num = filename.split('.')[0]
        colors[num] = dominant_color


KeyboardInterrupt: 

In [141]:
# COLOR THIEF make into pd df & merge w/ watercolors data
colors_df = pd.DataFrame.from_dict(colors, orient = 'index').reset_index()
colors_df.rename(columns={'index': 'accession_number', 0:'r', 1:'g', 2:'b'}, inplace=True)

watercolors = wc.merge(colors_df)
watercolors = watercolors[['accession_number', 'year', 'r', 'g', 'b' ]]
watercolors.to_csv('../cleaned_data/watercolors.csv')

In [73]:
# colorgram! 
# Get (colorgram) color for watercolor artworks
dir = '/Users/carolynliu/CAPP30239_FA22/final_project/pics'
colors = {}
for filename in os.listdir(dir):
    exp = re.compile(".jpg")
    if re.search(exp, filename):
        f = os.path.join(dir, filename)
        color_thief = ColorThief(f)
        # get 3 colors
        c = colorgram.extract(f, 3)
        num = filename.split('.')[0]
        colors[num] = c

# make into pd df & merge w/ watercolors data
colors_df = pd.DataFrame.from_dict(colors, orient = 'index').reset_index()

# split columns to make data easier to use
cols = ['rgb', 'prop']
for i in range(3):
    rgb = 'rgb' + str(i)
    prop = 'prop' + str(i)
    colors_df[rgb] = colors_df.apply(lambda row: tuple(row[i].rgb), axis=1)
    colors_df[prop] = colors_df.apply(lambda row: row[i].proportion, axis=1)

colors_df.rename(columns={'index': 'accession_number'}, inplace=True)

watercolors_cg = wc.merge(colors_df)
watercolors_cg = watercolors_cg[['accession_number', 'year', 'rgb0', 'prop0',	'rgb1',	'prop1', 'rgb2', 'prop2']]
watercolors_cg.to_csv('../cleaned_data/watercolors_cg.csv')